In [1]:
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.drift_detection.adwin import ADWIN

from float.feature_selection import FIRES           # online feature methods
from skmultiflow.data import FileStream             # create stream from file
from sklearn.model_selection import ParameterGrid   # hyperparameter combinations

from prequential import run_prequential

In [2]:
data_loader = FileStream(filepath='../out.csv')

In [3]:
# base arf classifier
arf = AdaptiveRandomForestClassifier()

In [4]:
ref_sample, _ = data_loader.next_sample(50)
data_loader.reset()

In [5]:
parameters = {
    'penalty_s': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1],    # default=0.01
    'penalty_r': [0.01, 0.1, 1],                # default=0.01
    'lr_mu': [0.01, 0.025, 0.1, 1],         # default=0.01
    'lr_sigma': [0.01, 0.025, 0.1, 1],      # default=0.01    
}

fires_grid = ParameterGrid(parameters)

In [6]:
best_params = None
best_score = 0
best_metrics = []

for params in fires_grid:
    print(f"Evaluating parameters: {params}")
    data_loader.restart()

    accuracy, precision, recall, f1, avg_processing_time = run_prequential(
        classifier=arf,
        stream=data_loader,
        feature_selector = FIRES(
                **params,
                n_total_features=data_loader.n_features,
                n_selected_features=10,
                classes=data_loader.target_values,
                baseline='gaussian',
                ref_sample=ref_sample
        ),
    )

    print(f"Accuracy: {accuracy:.6f}, Precision: {precision:.6f}, Recall: {recall:.6f}, F1: {f1:.6f}")
    print(f"Average processing time: {avg_processing_time}")

    if (accuracy + precision + recall + f1) / 4 > best_score:
        best_params = params
        best_score = (accuracy + precision + recall + f1) / 4
        best_metrics = [accuracy, precision, recall, f1]
        

print("Grid Search Completed.")
print(f"Best Parameters: {best_params}")
print(f"Best Metrics: {best_metrics}")

Evaluating parameters: {'lr_mu': 0.01, 'lr_sigma': 0.01, 'penalty_r': 0.01, 'penalty_s': 1e-05}
Accuracy: 0.998000, Precision: 0.997321, Recall: 0.998492, F1: 0.997906
Average processing time: 0.00927519624400001
Evaluating parameters: {'lr_mu': 0.01, 'lr_sigma': 0.01, 'penalty_r': 0.01, 'penalty_s': 0.0001}
Accuracy: 0.998300, Precision: 0.998532, Recall: 0.997905, F1: 0.998219
Average processing time: 0.008975489231999829
Evaluating parameters: {'lr_mu': 0.01, 'lr_sigma': 0.01, 'penalty_r': 0.01, 'penalty_s': 0.001}
Accuracy: 0.998040, Precision: 0.998114, Recall: 0.997779, F1: 0.997946
Average processing time: 0.009074551249999993
Evaluating parameters: {'lr_mu': 0.01, 'lr_sigma': 0.01, 'penalty_r': 0.01, 'penalty_s': 0.01}
Accuracy: 0.998240, Precision: 0.997948, Recall: 0.998366, F1: 0.998157
Average processing time: 0.00911095740600048
Evaluating parameters: {'lr_mu': 0.01, 'lr_sigma': 0.01, 'penalty_r': 0.01, 'penalty_s': 0.1}
Accuracy: 0.997720, Precision: 0.997278, Recall: 0.9

KeyboardInterrupt: 

In [ ]:
print(best_params)

In [ ]:
print(best_metrics)

In [ ]:
print(len(fires_grid))